In [3]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

<h1>Loading database</h1>

In [4]:
from pypots.benchpots.datasets import preprocess_physionet2012
physionet2012_dataset = preprocess_physionet2012(subset="all", rate=0.1)

2024-10-29 21:38:29 [INFO]: You're using dataset physionet_2012, please cite it properly in your work. You can find its reference information at the below link: 
https://github.com/WenjieDu/TSDB/tree/main/dataset_profiles/physionet_2012
2024-10-29 21:38:29 [INFO]: Dataset physionet_2012 has already been downloaded. Processing directly...
2024-10-29 21:38:29 [INFO]: Dataset physionet_2012 has already been cached. Loading from cache directly...
2024-10-29 21:38:30 [INFO]: Loaded successfully!


<h1>Training data</h1>

In [5]:
train_X = physionet2012_dataset['train_X']

In [21]:
distribution_gender = train_X['Gender'].value_counts(normalize=True) * 100
distribution_gender # 0: female, or 1: male

Gender
 1.0    55.494720
 0.0    44.374919
-1.0     0.130361
Name: proportion, dtype: float64

In [18]:
distribution_ICUType = train_X['ICUType'].value_counts(normalize=True) * 100
distribution_ICUType

ICUType
3.0    35.849303
4.0    28.327467
2.0    21.079390
1.0    14.743840
Name: proportion, dtype: float64

In [33]:
#train_X pega 60% de 12000
train_X['Age'].value_counts()

Age
90.0    13742
78.0    10000
77.0     9246
76.0     8461
71.0     8323
        ...  
31.0     1012
18.0      684
17.0      401
16.0      139
15.0       46
Name: count, Length: 76, dtype: int64

In [34]:
age_uniques = train_X.groupby('RecordID')['Age'].first().reset_index()

KeyError: 'RecordID'

In [28]:
first_values = train_X['Age'][::48] #first value of "Age" at every 48th index
total_quantity_of_age_occurrences = first_values.count()
total_quantity_of_age_occurrences

7671

In [26]:
more_than_or_equal_to_65 = train_X[train_X['Age'] >= 65] 
distribution_age_group_one = more_than_or_equal_to_65['Age'].count() 
distribution_age_group_one

190728

In [20]:
less_than_65 = train_X[train_X['Age'] < 65] 
distribution_age_group_two = less_than_65['Age'].value_counts(normalize=True) * 100
distribution_age_group_two

Age
61.0    4.745802
64.0    4.706018
63.0    4.374274
62.0    4.254282
58.0    4.242731
59.0    4.065630
56.0    3.937938
60.0    3.897512
54.0    3.590793
55.0    3.585018
57.0    3.580527
51.0    3.167290
47.0    3.068473
49.0    3.026122
50.0    2.947197
52.0    2.924097
53.0    2.820788
45.0    2.594278
48.0    2.507010
46.0    2.302317
42.0    2.070032
43.0    1.989823
44.0    1.940414
40.0    1.904481
41.0    1.598404
37.0    1.308368
39.0    1.259601
35.0    1.255751
38.0    1.242276
36.0    1.212759
34.0    1.137683
33.0    0.954165
23.0    0.901548
27.0    0.900265
22.0    0.875240
30.0    0.864973
28.0    0.838023
26.0    0.832890
24.0    0.821339
19.0    0.783481
25.0    0.715464
20.0    0.713539
21.0    0.705197
29.0    0.700705
32.0    0.671188
31.0    0.649371
18.0    0.438903
17.0    0.257310
16.0    0.089192
15.0    0.029517
Name: proportion, dtype: float64

<h1>Validation data</h1>

In [15]:
validation_X = physionet2012_dataset['val_X']

<h1>Test data</h1>

In [16]:
test_X = physionet2012_dataset['test_X']